In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('../notebook/data/EHR.csv')
df.head(10)

,Patient_ID,Age,Gender,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment,Response_to_Treatment,Survival_Status
0,c044501a-43ca-4a0c-8b8b-991439ba1b6a,52,Female,5.08,Benign,Positive,Surgery,No Response,Survived
1,b8900c4c-1232-4084-9432-5d02eba74d20,32,Female,0.80,Benign,Negative,Surgery,Complete Response,Survived
2,3004e2bc-8037-49cb-a542-d5612b73beab,70,Female,9.56,Benign,Positive,Radiation Therapy,Complete Response,Deceased
3,1df86af7-6745-4dea-b127-cbc9915079fc,21,Female,3.07,Malignant,Negative,Surgery,Partial Response,Survived
4,128e00c3-72e3-4031-a7f4-1165d7199cce,62,Male,7.17,Malignant,Positive,Radiation Therapy,Complete Response,Deceased
5,2b3cc8d5-f2f7-4ce2-be51-f5f489b53244,60,Female,8.31,Benign,Negative,Radiation Therapy,Complete Response,Deceased
6,2f8c5926-bedb-418e-84ed-8b05412e495f,34,Female,0.66,Benign,Negative,Chemotherapy,Complete Response,Survived
7,315f3ae6-b44c-42a8-9ce3-deb728a662f9,69,Female,2.20,Malignant,Negative,Surgery,Partial Response,Survived
8,494e30bf-b2e5-46b1-8323-ec4b1ef6ff1b,49,Male,6.42,Benign,Negative,Chemotherapy,No Response,Deceased
9,b55064cc-82e5-4d22-801a-32c2a9106a44,80,Female,2.04,Malignant,Negative,Chemotherapy,Partial Response,Deceased


In [4]:
df= df.drop(columns=["Patient_ID"])

In [5]:
X = df.drop(columns=["Survival_Status"])
y = df["Survival_Status"]

In [ ]:
X.head(10)

In [ ]:
y.head(10)

In [17]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
lbl_transformer = LabelEncoder()
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),        
    ]
)

NameError: name 'X' is not defined

In [7]:
y = lbl_transformer.fit_transform(df["Survival_Status"])

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)


In [12]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [9]:
X_train.shape

(14000, 7)

In [ ]:
y_train.shape

In [10]:
from sklearn.metrics import accuracy_score

In [13]:
from sklearn.tree import DecisionTreeClassifier
## base DTREE Model
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("training acc",accuracy_score(y_train,y_train_pred))
print("testing acc",accuracy_score(y_test,y_test_pred))
print("score",model.score)

training acc 0.9992857142857143
testing acc 0.504
score <bound method ClassifierMixin.score of DecisionTreeClassifier()>


### hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
dtree_model = DecisionTreeClassifier()
param_grid={
    'criterion':["gini","entropy","log_loss"],
    "splitter":["best","random"],
    "max_depth":[None,2,5,10, 20, 30, 40,],
    'max_features': [None, 'sqrt', 'log2'] , # Number of features to consider when looking for the best split
    "min_samples_split":[2,4,5,8,10],
    # "min_samples_leaf":[0,1,2,3,4,5]
}
gs = GridSearchCV(estimator=dtree_model,param_grid=param_grid,cv=8,n_jobs=-1,verbose=2)
gs.fit(X_train,y_train)


In [21]:
print("best params : " , gs.best_params_)
print("best score : " , gs.best_score_)

best params :  {'criterion': 'gini', 'max_depth': 2, 'max_features': None, 'min_samples_split': 8, 'splitter': 'random'}
best score :  0.5102142857142857


In [22]:
model1 = DecisionTreeClassifier(criterion='gini',max_depth=2,max_features=None,min_samples_split=8,splitter='random')
model1.fit(X_train,y_train)
y_train_pred = model1.predict(X_train)
y_test_pred = model1.predict(X_test)
print("training  : ", accuracy_score(y_train,y_train_pred))
print("tetsing  : ", accuracy_score(y_test,y_test_pred))

training  :  0.5081428571428571
tetsing  :  0.49733333333333335


### use bagging

In [16]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth = 5)
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("training acc",accuracy_score(y_train,y_train_pred))
print("testing acc",accuracy_score(y_test,y_test_pred))
print("score",model.score)

training acc 0.5607142857142857
testing acc 0.5078333333333334
score <bound method ClassifierMixin.score of RandomForestClassifier(max_depth=5)>


not overfitting anymore. but accuracy is low

In [ ]:
model = RandomForestClassifier()
param_grid={
    'n_estimators':[80,100,110],
    'criterion':["gini","entropy","log_loss"],
    "max_depth":[None,2,5,8,10,15],
    'max_features': [None, 'sqrt', 'log2'] , # Number of features to consider when looking for the best split
    "min_samples_split":[2,4,5,8,10],
    "oob_score" :[True,False],
    "bootstrap" :[True,False]
}
gs = GridSearchCV(estimator=model,param_grid=param_grid,cv=5,n_jobs=-1,verbose=2)
gs.fit(X_train,y_train)

In [ ]:
gs.best_score_

In [ ]:
x = df.iloc[:,:-1]
x

In [ ]:
ohc = OneHotEncoder();
x = ohc.fit_transform(x)

even after using bagging the performance does not increases by much

In [4]:
dfec = pd.read_csv("../notebook/data/EncodedEHRData.csv")

In [25]:
x = dfec.iloc[:,1:]

In [26]:
x.corr()

,Age,Gender,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment,Response_to_Treatment,Survival_Status
Age,1.000000,0.002081,0.003918,-0.003325,-0.000569,0.007672,-0.010751,0.007234
Gender,0.002081,1.000000,0.009110,-0.001765,-0.004931,-0.009952,-0.000929,-0.008128
Tumor_Size(cm),0.003918,0.009110,1.000000,0.010898,-0.001929,0.000009,-0.000909,0.010512
Tumor_Type,-0.003325,-0.001765,0.010898,1.000000,0.003595,-0.004416,0.009027,0.000877
Biopsy_Result,-0.000569,-0.004931,-0.001929,0.003595,1.000000,-0.006274,0.005732,0.009890
Treatment,0.007672,-0.009952,0.000009,-0.004416,-0.006274,1.000000,0.009800,0.002964
Response_to_Treatment,-0.010751,-0.000929,-0.000909,0.009027,0.005732,0.009800,1.000000,-0.013999
Survival_Status,0.007234,-0.008128,0.010512,0.000877,0.009890,0.002964,-0.013999,1.000000


all the features are almost equally co-related upto same extent. 
but Gender & Response_to_Treatment are negetively co-related.

In [20]:
from sklearn.preprocessing import LabelEncoder
lbl_transformer = LabelEncoder()

In [16]:
df = pd.read_csv('../notebook/data/EHR.csv')

In [38]:
df.head(5)

,Patient_ID,Age,Gender,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment,Response_to_Treatment,Survival_Status
0,c044501a-43ca-4a0c-8b8b-991439ba1b6a,52,0,5.08,0,1,2,1,Survived
1,b8900c4c-1232-4084-9432-5d02eba74d20,32,0,0.80,0,0,2,0,Survived
2,3004e2bc-8037-49cb-a542-d5612b73beab,70,0,9.56,0,1,1,0,Deceased
3,1df86af7-6745-4dea-b127-cbc9915079fc,21,0,3.07,1,0,2,2,Survived
4,128e00c3-72e3-4031-a7f4-1165d7199cce,62,1,7.17,1,1,1,0,Deceased


In [29]:
df["Gender"] = lbl_transformer.fit_transform(df["Gender"])
df["Treatment"] = lbl_transformer.fit_transform(df["Treatment"])
df["Response_to_Treatment"] = lbl_transformer.fit_transform(df["Response_to_Treatment"])
df["Biopsy_Result"] = lbl_transformer.fit_transform(df["Biopsy_Result"])
df["Tumor_Type"] = lbl_transformer.fit_transform(df["Tumor_Type"])

In [30]:
df_gender_eliminated = df.drop(columns=["Gender","Patient_ID","Survival_Status"])
df_rtt_eliminated = df.drop(columns=["Response_to_Treatment","Patient_ID","Survival_Status"])
df_both_eliminated = df.drop(columns=["Gender","Response_to_Treatment","Patient_ID","Survival_Status"])

In [32]:
y = dfec.iloc[:,-1]
y = lbl_transformer.fit_transform(y)

In [33]:
df_gender_eliminated.head(5)

,Age,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment,Response_to_Treatment
0,52,5.08,0,1,2,1
1,32,0.80,0,0,2,0
2,70,9.56,0,1,1,0
3,21,3.07,1,0,2,2
4,62,7.17,1,1,1,0


In [34]:
df_rtt_eliminated.head(5)

,Age,Gender,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment
0,52,0,5.08,0,1,2
1,32,0,0.80,0,0,2
2,70,0,9.56,0,1,1
3,21,0,3.07,1,0,2
4,62,1,7.17,1,1,1


In [35]:
df_both_eliminated.head(5)

,Age,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment
0,52,5.08,0,1,2
1,32,0.80,0,0,2
2,70,9.56,0,1,1
3,21,3.07,1,0,2
4,62,7.17,1,1,1


In [37]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [61]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_both_eliminated,y,test_size=0.3,random_state=42)

In [44]:
from sklearn.metrics import accuracy_score

In [62]:
X_train.head(5)

,Age,Tumor_Size(cm),Tumor_Type,Biopsy_Result,Treatment
17218,62,2.74,0,1,1
15188,45,6.41,0,1,2
11295,45,2.90,0,0,2
19772,70,1.53,1,1,2
13072,40,2.10,0,0,0


In [66]:
from sklearn.tree import DecisionTreeClassifier
dtree_clsf_model = DecisionTreeClassifier(criterion='gini',max_depth=2,max_features=None,min_samples_split=8,splitter='random')
dtree_clsf_model.fit(X_train,y_train)
train_pred = dtree_clsf_model.predict(X_train)
test_pred = dtree_clsf_model.predict(X_test)

print("After Eliminating the Response to Treatment Feature")
print("Training acc_score  : ",accuracy_score(y_train,train_pred))
print("Testing acc_score  : ",accuracy_score(y_test,test_pred))

After Eliminating the Response to Treatment Feature
Training acc_score  :  0.5087857142857143
Testing acc_score  :  0.502


### After Eliminating the Gender Feature

Training acc_score  :  0.5081428571428571

Testing acc_score  :  0.49733333333333335


### After Eliminating the Response to Treatment Feature

Training acc_score  :  0.5085714285714286

Testing acc_score  :  0.506


### After Eliminating the Response to Treatment Feature

Training acc_score  :  0.5087857142857143

Testing acc_score  :  0.502


## Not much improvement!!!